In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
from keras.optimizers import Adam


In [4]:
# #Line used to load dataset in google colab
from google.colab import files

uploaded = files.upload()

Saving lotto_polonia_2.csv to lotto_polonia_2.csv


In [5]:
# Load the dataset
df = pd.read_csv('lotto_polonia_2.csv', delimiter=';')
df.head()

,Numer,Dzien,Miesiac,Rok,L1,L2,L3,L4,L5,L6,...,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20
0,1,18,3,1996,4,9,10,16,21,22,...,35,41,42,48,62,66,68,73,76,78
1,2,19,3,1996,6,12,15,19,28,33,...,49,59,62,63,64,67,69,71,75,77
2,3,21,3,1996,2,4,6,7,15,16,...,28,45,48,52,54,69,72,73,75,77
3,4,22,3,1996,3,8,15,17,19,25,...,36,38,44,49,60,61,64,67,68,75
4,5,25,3,1996,2,10,11,14,18,22,...,42,44,45,55,60,61,66,67,75,79


In [6]:
#Check the last lines from the data set
df.tail()

,Numer,Dzien,Miesiac,Rok,L1,L2,L3,L4,L5,L6,...,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20
14493,14494,29,3,2023,1,3,4,6,8,12,...,28,36,45,55,56,63,64,69,70,72
14494,14495,29,3,2023,4,9,12,13,16,17,...,37,39,40,42,50,51,59,61,71,76
14495,14496,30,3,2023,1,3,5,6,11,13,...,36,44,50,53,61,63,67,69,79,80
14496,14497,30,3,2023,3,5,6,10,24,29,...,49,55,56,60,62,67,71,72,73,79
14497,14498,31,3,2023,2,4,6,10,17,23,...,45,50,53,58,61,66,75,78,79,80


In [7]:
#Print the shape of the dataset
df.shape

(14498, 24)

In [8]:
#Info of the data set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14498 entries, 0 to 14497
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Numer    14498 non-null  int64
 1   Dzien    14498 non-null  int64
 2   Miesiac  14498 non-null  int64
 3   Rok      14498 non-null  int64
 4   L1       14498 non-null  int64
 5   L2       14498 non-null  int64
 6   L3       14498 non-null  int64
 7   L4       14498 non-null  int64
 8   L5       14498 non-null  int64
 9   L6       14498 non-null  int64
 10  L7       14498 non-null  int64
 11  L8       14498 non-null  int64
 12  L9       14498 non-null  int64
 13  L10      14498 non-null  int64
 14  L11      14498 non-null  int64
 15  L12      14498 non-null  int64
 16  L13      14498 non-null  int64
 17  L14      14498 non-null  int64
 18  L15      14498 non-null  int64
 19  L16      14498 non-null  int64
 20  L17      14498 non-null  int64
 21  L18      14498 non-null  int64
 22  L19      14498 non-nul

In [9]:
#Prints the min, max, median and std
df.describe()

,Numer,Dzien,Miesiac,Rok,L1,L2,L3,L4,L5,L6,...,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20
count,14498.000000,14498.00000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,...,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000,14498.000000
mean,7249.500000,15.74969,6.528901,2011.874603,3.903366,7.800731,11.658022,15.526142,19.387019,23.254449,...,42.394951,46.303145,50.201200,54.074838,57.942613,61.784798,65.612360,69.441509,73.256173,77.136984
std,4185.356436,8.79544,3.456790,7.146657,3.196229,4.396325,5.236363,5.848249,6.310839,6.723648,...,7.420929,7.384897,7.245258,7.027198,6.717572,6.337622,5.817791,5.182500,4.334679,3.149009
min,1.000000,1.00000,1.000000,1996.000000,1.000000,2.000000,3.000000,4.000000,5.000000,7.000000,...,16.000000,21.000000,23.000000,27.000000,31.000000,33.000000,35.000000,40.000000,43.000000,49.000000
25%,3625.250000,8.00000,4.000000,2007.000000,2.000000,4.000000,8.000000,11.000000,15.000000,18.000000,...,37.000000,41.000000,45.000000,49.000000,54.000000,58.000000,62.000000,66.000000,71.000000,76.000000
50%,7249.500000,16.00000,7.000000,2013.000000,3.000000,7.000000,11.000000,15.000000,19.000000,23.000000,...,42.000000,46.000000,50.000000,54.000000,58.000000,62.000000,66.000000,70.000000,74.000000,78.000000
75%,10873.750000,23.00000,10.000000,2018.000000,5.000000,10.000000,15.000000,19.000000,23.000000,28.000000,...,48.000000,51.000000,55.000000,59.000000,63.000000,66.000000,70.000000,73.000000,77.000000,79.000000
max,14498.000000,31.00000,12.000000,2023.000000,31.000000,34.000000,44.000000,45.000000,46.000000,51.000000,...,67.000000,69.000000,71.000000,73.000000,74.000000,76.000000,77.000000,78.000000,79.000000,80.000000


In [10]:
# Drop unnecessary columns
df.drop(['Numer', 'Dzien', 'Miesiac', 'Rok'], axis=1, inplace=True)
df.head()

,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19,L20
0,4,9,10,16,21,22,23,26,27,34,35,41,42,48,62,66,68,73,76,78
1,6,12,15,19,28,33,35,39,44,48,49,59,62,63,64,67,69,71,75,77
2,2,4,6,7,15,16,17,19,20,26,28,45,48,52,54,69,72,73,75,77
3,3,8,15,17,19,25,30,33,34,35,36,38,44,49,60,61,64,67,68,75
4,2,10,11,14,18,22,26,27,29,30,42,44,45,55,60,61,66,67,75,79


In [11]:
# Standardize the dataset
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

transformed_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.030235,0.272798,-0.316647,0.081028,0.255598,-0.186579,-0.580812,-0.679808,-1.057296,-0.615858,-0.996534,-0.718132,-1.131979,-0.864505,0.604017,0.665131,0.410417,0.686660,0.633016,0.274069
1,0.655994,0.955210,0.638247,0.594020,1.364839,1.449494,1.134443,1.118125,1.259779,1.268914,0.890088,1.719359,1.628542,1.270132,0.901754,0.822924,0.582310,0.300732,0.402310,-0.043502
2,-0.595524,-0.864554,-1.080562,-1.457947,-0.695180,-1.078983,-1.438440,-1.647925,-2.011386,-1.692870,-1.939844,-0.176467,-0.303823,-0.295268,-0.586931,1.138511,1.097987,0.686660,0.402310,-0.043502
3,-0.282645,0.045328,0.638247,0.252026,-0.061328,0.259623,0.419753,0.288310,-0.103206,-0.481231,-0.861775,-1.124380,-0.855927,-0.722196,0.306280,-0.123836,-0.277153,-0.471123,-1.212628,-0.678645
4,-0.595524,0.500269,-0.125668,-0.260966,-0.219791,-0.186579,-0.151999,-0.541505,-0.784699,-1.154364,-0.053223,-0.311883,-0.717901,0.131659,0.306280,-0.123836,0.066632,-0.471123,0.402310,0.591640


In [12]:
# Prepare input and output data
number_of_rows = df.values.shape[0]
window_lenght = 21
number_of_features = df.values.shape[1]

X = np.empty([number_of_rows - window_lenght, window_lenght, number_of_features], dtype=float)
y = np.empty([number_of_rows - window_lenght, number_of_features], dtype=float)

for i in range(0, number_of_rows - window_lenght):
    X[i] = transformed_df.iloc[i : i + window_lenght, 0 : number_of_features]
    y[i] = transformed_df.iloc[i + window_lenght : i + window_lenght + 1, 0 : number_of_features]

In [13]:
# Define the model architecture
model = Sequential()
model.add(Bidirectional(LSTM(400, input_shape = (window_lenght, number_of_features), return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(400, input_shape = (window_lenght, number_of_features), return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(400, input_shape = (window_lenght, number_of_features), return_sequences = True)))
model.add(Bidirectional(LSTM(400, input_shape = (window_lenght, number_of_features), return_sequences = False)))
model.add(Dense(160))
model.add(Dense(number_of_features))

In [14]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0003), loss ='mse', metrics=['accuracy'])

In [15]:
# Train the model
model.fit(x=X, y=y, batch_size=160, epochs=210, verbose=2)

Epoch 1/210
91/91 - 27s - loss: 1.0029 - accuracy: 0.0587 - 27s/epoch - 297ms/step
Epoch 2/210
91/91 - 9s - loss: 1.0011 - accuracy: 0.0694 - 9s/epoch - 103ms/step
Epoch 3/210
91/91 - 9s - loss: 1.0005 - accuracy: 0.0629 - 9s/epoch - 104ms/step
Epoch 4/210
91/91 - 10s - loss: 1.0003 - accuracy: 0.0662 - 10s/epoch - 106ms/step
Epoch 5/210
91/91 - 10s - loss: 1.0000 - accuracy: 0.0649 - 10s/epoch - 107ms/step
Epoch 6/210
91/91 - 10s - loss: 0.9999 - accuracy: 0.0700 - 10s/epoch - 105ms/step
Epoch 7/210
91/91 - 9s - loss: 0.9996 - accuracy: 0.0672 - 9s/epoch - 104ms/step
Epoch 8/210
91/91 - 9s - loss: 0.9988 - accuracy: 0.0697 - 9s/epoch - 102ms/step
Epoch 9/210
91/91 - 9s - loss: 0.9988 - accuracy: 0.0709 - 9s/epoch - 102ms/step
Epoch 10/210
91/91 - 9s - loss: 0.9985 - accuracy: 0.0695 - 9s/epoch - 102ms/step
Epoch 11/210
91/91 - 9s - loss: 0.9967 - accuracy: 0.0744 - 9s/epoch - 102ms/step
Epoch 12/210
91/91 - 9s - loss: 0.9960 - accuracy: 0.0670 - 9s/epoch - 103ms/step
Epoch 13/210
91/9

In [16]:
# Make a prediction
to_predict = df.tail(21)
to_predict = np.array(to_predict)
scaled_to_predict = scaler.transform(to_predict)
y_pred = model.predict(np.array([scaled_to_predict]))
predicted_numbers = scaler.inverse_transform(y_pred).astype(int)[0]
print('The predicted numbers in the last lottery game are:', predicted_numbers)

1/1 [==============================] - 2s 2s/step
The predicted numbers in the last lottery game are: [ 2  5  8 12 16 22 23 28 31 42 45 50 52 55 58 63 70 75 77 79]


In [17]:
#The withdraw numbers
numbers1 = np.array([1,9,11,13,22,23,25,31,33,42,45,51,57,59,60,63,65,70,74,77])
numbers1
# numbers1 = np.array([2,4,6,10,17,23,24,26,30,44,45,50,53,58,61,66,75,78,79,80])
# numbers1

array([ 1,  9, 11, 13, 22, 23, 25, 31, 33, 42, 45, 51, 57, 59, 60, 63, 65,
       70, 74, 77])

In [18]:
#Check how many numbers are the same with the withdraw ones
np.intersect1d(scaler.inverse_transform(y_pred).astype(int)[0], numbers1)

array([22, 23, 31, 42, 45, 63, 70, 77])

In [19]:
# Calculate the probability of each predicted number
probabilities = []
for number in predicted_numbers:
    occurrences = df.tail(20)[df.tail(20) == number].count().sum()  # Count the number of times the predicted number has occurred in the next 20 numbers
    probability = occurrences / len(df.tail(20)) * 100  # Calculate the probability as a percentage among the next 20 numbers
    probabilities.append(probability)

In [20]:
# Normalize the probabilities so that they add up to 100%
probabilities = [p/sum(probabilities)*100 for p in probabilities]

In [24]:
# Print the predicted numbers and their corresponding probabilities
for i in range(len(predicted_numbers)):
    print(f"The predicted number {predicted_numbers[i]} has a probability of {probabilities[i]:.2f}% to be drawn next.")

The predicted number 2 has a probability of 3.57% to be drawn next.
The predicted number 5 has a probability of 5.36% to be drawn next.
The predicted number 8 has a probability of 1.79% to be drawn next.
The predicted number 12 has a probability of 4.46% to be drawn next.
The predicted number 16 has a probability of 3.57% to be drawn next.
The predicted number 22 has a probability of 4.46% to be drawn next.
The predicted number 23 has a probability of 4.46% to be drawn next.
The predicted number 28 has a probability of 3.57% to be drawn next.
The predicted number 31 has a probability of 5.36% to be drawn next.
The predicted number 42 has a probability of 4.46% to be drawn next.
The predicted number 45 has a probability of 8.04% to be drawn next.
The predicted number 50 has a probability of 8.93% to be drawn next.
The predicted number 52 has a probability of 2.68% to be drawn next.
The predicted number 55 has a probability of 4.46% to be drawn next.
The predicted number 58 has a probabi